## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Привет! Поздравляю тебя с первым твоим проектом и спасибо, что вовремя сдал задание:) Ты проделал большую работу. Далее в файле мои коммефнтарии ты сможешь найти в ячейках, аналогичных данной ( если рамки комментария зелёные - всё сделано правильно; жёлтые - есть замечания, но не критично; красные - нужно переделать). Не удаляй эти комментарии и постарайся учесть их в ходе выполнения проекта. 

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [284]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter
credit_score = pd.read_csv('/datasets/data.csv')
#посмотрим общую информацию
credit_score.info()
#таблица визуализация (посмотрим заголовки)
credit_score.head(15)
#таблица оказалась огромной (посмотрим хвост)
credit_score.tail(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


### Вывод

Что увидел: 1) отрицательные значения в детях и днях стажа 2) отсутствую строки в днях стажа и доходе 3) в столбцe education разный регистр букв 4) в чём (валюта) месячный доход (не уверен важно ли это) 5) максимально детей 20, хотя нет значений 10 или 15 6) есть возраст = 0 (наверное это отсутствие детей)

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Хорошее начало, так же, для большего понимания данных можно было бы использовать метод `describe`


### Шаг 2. Предобработка данных

### Обработка пропусков

In [285]:
# Посчитаем суммарное количество пропусков
credit_score.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [286]:
# Замена пропусков
credit_score['days_employed'] = credit_score['days_employed'].fillna(0)
credit_score['total_income'] = credit_score['total_income'].fillna(credit_score['total_income'].median())
# Что получилось
credit_score.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

Стаж и доход одинаковы по пропускам, возможно данные не корректны, либо эти люди не работают. Надо посмотреть ещё раз содержимое столбцов.От пропусков избавился забив нулями и медианой, по совету преподавателя в слаке). 

<div style="border:solid  red  2px; padding: 20px"> <h1 style="color: red ; margin-bottom:20px">Комментарий наставника</h1>

Тут не достаточно заполнить просто медианой, средним или нулём, требуется более обоснованное заполнение пропусков исходя из имеющихся данных



#По коментарию про заполнение пропусков, сначала я хотел оба важных стобцах заполнить медианой, 
#но тогда в итоговом анализе цифры будут другие и анализ будет более положительным. 
#Поэтому выбрал такой метод, для стажа нули, чтобы уровнять шансы, а для дохода медиана, 
#чтобы не было людей которые из-за ошибки заполнения или некорреткной выгрузки данных, 
#не оказались в "черном списке" по одобрению кредита, в целом же медианные значения в обоих столбцах сделают итог более позитивным для клиента 
#но в возможной перспективе убыточной для банка

#"требуется более обоснованное заполнение пропусков исходя из имеющихся данных" 
#мне нужно переработать код или достаточно пояснения выше?

<div style="border:solid  orange  2px; padding: 20px"> <h1 style="color: orange ; margin-bottom:20px">Комментарий наставника</h1>

Обоснования твои вижу, но всё же на будущее - не пытайся как-то подстроить цифры и подвести под какой-то результат, лучше сделать всё точно и по честному

### Замена типа данных

In [288]:
#Работа с отрицательным стажем
#credit_score['days_employed'] = pd.to_numeric(credit_score['days_employed'], errors='ignore')
#Нумерик не получился((
#В слаке подсказали делать через abs (яндексим что за зверь)
credit_score['days_employed'] = credit_score['days_employed'].abs()
credit_score['days_employed'].head()

0      8437.673028
1      4024.803754
2      5623.422610
3      4124.747207
4    340266.072047
Name: days_employed, dtype: float64

In [289]:
#попробуем перевести значение в года
credit_score['years_employed'] = (credit_score['days_employed'] / 365).astype('int')
credit_score['years_employed'].head(15)

0      23
1      11
2      15
3      11
4     932
5       2
6       7
7       0
8      18
9       5
10     11
11      2
12      0
13      5
14      5
Name: years_employed, dtype: int64

In [290]:
#932 года стажа... ПФР не обрадуется... Думаю опять кто-то или что-то не правильно выгрузило данные

In [291]:
#Исправляем образование (регистр)
credit_score['education'] = credit_score['education'].str.lower()
credit_score['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [292]:
#теперь возраст, у нас есть 0-е значение
credit_score['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

In [293]:
credit_score[credit_score['dob_years'] == 0]['dob_years'].count()

101

In [294]:
#101 человек с таковым значением.

In [295]:
#в категоризации выяснилось, что нужно поменять тип данных в "доходе"
credit_score['total_income'] = credit_score['total_income'].astype('int')
credit_score['total_income'].head()

0    253875
1    112080
2    145885
3    267628
4    158616
Name: total_income, dtype: int64

### Вывод

Стаж стал адекватным кроме огромного значения, пока нет идей что это.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Замена типа данных выполнена верно

### Обработка дубликатов

In [296]:
#Проверка наличия дублей
credit_score.duplicated().sum()

71

In [297]:
# Удалим дубли и сбросим индекс
credit_score = credit_score.drop_duplicates().reset_index(drop=True)

In [298]:
#Проверка
credit_score.duplicated().sum()

0

In [299]:
credit_score.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,932
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,2
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,7
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,0
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,18
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,5


In [300]:
#думаю исследования можно описать в этом блоке т.к. лемматизация это судя по всему будет промежуточным итогом
#По тренажёру мы не сможешь обойтись без классики - поиска уникальных значений
#Начнём со странных "отрицательных детей"
credit_score['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [301]:
#узнаем количество отрицательных значений
credit_score[credit_score['children'] == -1]['children'].count()

47

In [302]:
#надо заменить отрицательные значения
credit_score['children'] = credit_score['children'].replace(-1, 1)
credit_score[credit_score['children'] == -1]['children'].count()

0

In [303]:
#посчитаем максимальнео количество детей котрых 20
credit_score[credit_score['children'] == 20]['children'].count()
credit_score[credit_score['children'] == 20].groupby('dob_years')['dob_years'].count()

dob_years
0     1
21    1
23    1
24    1
25    1
26    1
27    2
29    2
30    3
31    2
32    2
33    2
34    3
35    2
36    2
37    4
38    1
39    1
40    4
41    2
42    3
43    2
44    2
45    3
46    3
48    1
49    3
50    3
51    1
52    1
53    1
54    1
55    1
56    5
57    1
59    2
60    1
61    1
62    1
64    1
69    1
Name: dob_years, dtype: int64

In [304]:
#в выборке нет нужного нам количества... Либо я что-то забыл (хотя код работает), либо учитывая число 20 и отсутствие например 10 или 15 это ошибочный ввод.
#наверное 20 это каким-то непонятным образом это всё таки кол-во 2, хотя как так вводилось число что добавился 0, неизвестно
credit_score['children'] = credit_score['children'].replace(20, 2)
#снова проверка
credit_score['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Вывод

С дублями расправились легко, по методу в тренажёре, дублирование записей учитывая свой опыт в excel происходит когда мержишь таблицы с одинаковой базой (основа для заполнения) но заполненой как попало и разными людьми, хотя, у нас csv, а люди таки обычно в xlsx выгружают, значит проблема в базе формирования, как вариант 1С.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Дубликаты найдены и обработаны верно,отлично


### Лемматизация

In [305]:
#в таблице есть цели кредита, они и будут основой для лемм
purposes_unique = credit_score['purpose'].unique()
print(purposes_unique)

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


In [306]:
#лемматизируем этот столбец, склеим методом join и расстановкой знаков разделения
m = Mystem()
string = '; '.join(purposes_unique)
lemmas = m.lemmatize(string)
from collections import Counter
#print(Counter(lemmas))
sorted(Counter(lemmas).items(), key = lambda pair: pair[1], reverse=True)

[(' ', 59),
 ('; ', 37),
 ('покупка', 10),
 ('недвижимость', 10),
 ('автомобиль', 9),
 ('образование', 9),
 ('жилье', 7),
 ('с', 5),
 ('операция', 4),
 ('на', 4),
 ('свой', 4),
 ('свадьба', 3),
 ('строительство', 3),
 ('получение', 3),
 ('высокий', 3),
 ('дополнительный', 2),
 ('для', 2),
 ('коммерческий', 2),
 ('жилой', 2),
 ('подержать', 2),
 ('заниматься', 2),
 ('сделка', 2),
 ('приобретение', 1),
 ('сыграть', 1),
 ('проведение', 1),
 ('семья', 1),
 ('собственный', 1),
 ('со', 1),
 ('профильный', 1),
 ('сдача', 1),
 ('ремонт', 1),
 ('\n', 1)]

### Вывод

Благодаря лемме, выявили топ запросов на кредит: недвижимость, автомобиль, образование, жилье

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>


Лемматизация проведена верно

### Категоризация данных

In [307]:
credit_score_pivot_z = credit_score.pivot_table(index=['income_type'], values='total_income', aggfunc='max')
print(credit_score_pivot_z.head())

             total_income
income_type              
безработный        202722
в декрете           53829
госслужащий        910451
компаньон         2265604
пенсионер          735103


In [308]:
#Учитывя Топ запросов на кредит, возьмём категории которые будут влиять на весь кредитный процесс будут следующие - доход, стаж, возраст, дети
def total_income_ktg(row):
    if row['total_income'] <= 50000:
        return 'нищий'
    elif 50000 < row['total_income'] <= 120000:
        return 'средний'
    elif 120000 < row['total_income'] < 1000000:
        return 'достаточный'
    else:
        return 'обеспеченный'

   
def days_employed_ktg(row):
    if row['years_employed'] <= 10:
        return 'стаж до 10 лет'
    elif 10 < row['years_employed'] <= 30:
        return 'стаж 10-30 лет'
    else:
        return 'стаж более 30 лет'

    
def dob_years_ktg(row):
    if row['dob_years'] < 30:
        return 'до 30 лет'
    elif 30 <= row['dob_years'] < 45:
        return '30-45 лет'
    elif 45 <= row['dob_years'] < 70:
        return '45-70 лет'
    else:
        return 'старше 70 лет'
    
    
def children_ktg(row):
    if row['children'] == 0:
        return 'нет детей'
    elif 1 <= row['children'] <= 2:
        return '1-2 ребенка'
    else:
        return 'больше 2-х'

In [309]:
credit_score['total_income'] = credit_score.apply(total_income_ktg, axis=1)
credit_score['years_employed'] = credit_score.apply(days_employed_ktg, axis=1)
credit_score['dob_years'] = credit_score.apply(dob_years_ktg, axis=1)
credit_score['children'] = credit_score.apply(children_ktg, axis=1)

In [310]:
credit_score.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1-2 ребенка,8437.673028,30-45 лет,высшее,0,женат / замужем,0,F,сотрудник,0,достаточный,покупка жилья,стаж 10-30 лет
1,1-2 ребенка,4024.803754,30-45 лет,среднее,1,женат / замужем,0,F,сотрудник,0,средний,приобретение автомобиля,стаж 10-30 лет
2,нет детей,5623.422610,30-45 лет,среднее,1,женат / замужем,0,M,сотрудник,0,достаточный,покупка жилья,стаж 10-30 лет
3,больше 2-х,4124.747207,30-45 лет,среднее,1,женат / замужем,0,M,сотрудник,0,достаточный,дополнительное образование,стаж 10-30 лет
4,нет детей,340266.072047,45-70 лет,среднее,1,гражданский брак,1,F,пенсионер,0,достаточный,сыграть свадьбу,стаж более 30 лет


### Вывод

Категоризация прошло успешно, таблица стала похожа на итоговые поиски

<div style="border:solid  red  2px; padding: 20px"> <h1 style="color: red ; margin-bottom:20px">Комментарий наставника</h1>


Требуется привести обоснованный выбор уровней зарплат на основе данных

In [ ]:
#Добавил сводную таблицу по роду деятельности и доходу, с значением аргумента, указанные ЗП я выбирал глазами, 
#но вот с помощью этого метода, можно наглядно посмотреть какие есть в массиве. Цифры брал приятные глазу и простые для elif.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Категоризация проведена верно

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [311]:
#В слаке советовали вывод итоговый делать через функцию, либо через "pivot"

In [312]:
credit_score_pivot_chl = credit_score.pivot_table(index=['children'], values='debt', aggfunc='mean')
print(credit_score_pivot_chl.head())

                 debt
children             
1-2 ребенка  0.092654
больше 2-х   0.081579
нет детей    0.075438


### Вывод

Зависить логичная, где нет детей или меньше, кредит возвращают охотней (в целом)

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [313]:
credit_score_pivot_fm = credit_score.pivot_table(index=['family_status'], values='debt', aggfunc='mean')
print(credit_score_pivot_fm.head())

                           debt
family_status                  
Не женат / не замужем  0.097509
в разводе              0.071130
вдовец / вдова         0.065693
гражданский брак       0.093471
женат / замужем        0.075452


### Вывод

Не женатые более склонны к некачественному исполнению обязательств

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [314]:
credit_score_pivot_ti = credit_score.pivot_table(index=['total_income'], values='debt', aggfunc='mean')
print(credit_score_pivot_ti.head())

                  debt
total_income          
достаточный   0.081459
нищий         0.061828
обеспеченный  0.080000
средний       0.081570


### Вывод

Жизнь и собственный опыт подтверждены этим кейсом, нищие более финансово дисциплинированы

- Как разные цели кредита влияют на его возврат в срок?

In [315]:
credit_score_pivot_p = credit_score.pivot_table(index=['purpose'], values='debt', aggfunc='mean')
print(credit_score_pivot_p.head())

                                debt
purpose                             
автомобили                  0.092050
автомобиль                  0.085020
высшее образование          0.088496
дополнительное образование  0.082609
жилье                       0.071207


### Вывод

Ещё одно подтверждение жизненных фактов, тем кто берёт займ на жильё (возможно первое собственное и к тому же единственное) более дисциплинированы в погашении

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Все выводы верны, код написан правильно, радует, что используешь сводные таблицы)


### Шаг 4. Общий вывод

Анализ полученной выгрузки от Банка, показал, что, данные не идеальны, содержат пропуски значений в важных столбцах, имеют неудобный тип, занесены с ошибками (регистр, числа), требуют упорядочивания и пересмотра. 
Беглый осмотр, показал, что люди, в выборке кредитного отдела, имеют разное количество детей (более 2-х), разборс дохода даже при сравнимом стаже и возрасте. 
Анализ заставил пересмотреть стаж (произведен перевод в года), изменить вывод дохода (для итогового анализа), сделать группировку по информации которая изначально имела неоднозначные сведения.
Путем вышеописанных действий, были важные для последующей работы с данными в банке, а именно: - люди имеющие не более 2-х детей или вовсе без оных возвращают средства более дисциплинированно в отличе от остальных, - люди не состоящие в браке также более финансово ответственны перед кредитором, -люди с низком достатком стараюстся выполнять свои кредитные обязательства более ответственно в отличие от людей с высоким доходом, -наиболее возвращаемые кредиты выдаются на приобретение собственного жилья.
Для банка рекомендуется произвести дополнительный контроль выгружаемой информации, что позволит снизить количество дубликатов и пропусков, указать работникам ответственным за наполнение БД, что от их действий зависит финансовая безопасность Банка и соответственно их финансовое благополучие.

<div style="border:solid  red  2px; padding: 20px"> <h1 style="color: red ; margin-bottom:20px">Комментарий наставника</h1>

Требуется написать общий вывод из проделанной работы со всеми результатами и гипотезами, которые были выведены в результате.Так же в выводе можешь указать рекомендации по использованию данного анализа в будущем и так же можно приветси цифры, полученные в ходе работы, для большей наглядности


<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Точные и обоснованные выводы, супер)

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.

<div style="border:solid  orange  2px; padding: 20px"> <h1 style="color: orange ; margin-bottom:20px">Комментарий наставника</h1>

Работа хорошая, но стоит исправить обозначенные неточности, буду ждать исправленный вариант, удачи!)


In [ ]:
#Спасибо, но до хорошей работы мне далеко, т.к. первый день 
#работы я посвятил беглому осмотру самого начала курса, и судорожному стуку клавиш, в пусто блокноте Юпитера
#проверяя его работоспособность.
#Не уверен, можно ли вот так отвечать не по теме проекта, но я надеюсь правильно понял замечания. 

<div style="border:solid  green  2px; padding: 20px"> <h1 style="color: green ; margin-bottom:20px">Комментарий наставника</h1>

#### Код

Всё отлично. Из того, что очень порадовало - соблюдена структура проекта, шаги из задания обозначены и выполнены последовательно, код написан аккуратно, используются комментарии к коду, быстро можно понять, какие операции выполняют сложные конструкции. В качестве совета предлагаю глубже изучить и начать чаще применять конструкцию try-except в решении задачи — это улучшит отказоустойчивость кода и обезопасит код от поломок в будущем, а так же изучить средства для построения изображений в Python для более глубокого понимания данных.
#### Выводы

У тебя отлично получается анализировать сложные данные, выдвигать корректные гипотезы и проверять свои выводы на возможность соответствия реальности. Видно глубокое понимание сути проведённого анализа. Было очень интересно проверять твой проект и следить за твоей мыслью, так держать!)


<div style="border:solid  green  2px; padding: 20px"> <h1 style="color: green ; margin-bottom:20px">Комментарий наставника</h1>
Работа принята!) Очень рад, что у тебя всё получилось и надеюсь тебе понравится весь курс, будь усерден и у тебя всё получится! Удачи в будущих проектах)

In [ ]:
#Спасибо за отзыв. Надеюсь, что дальше будет ещё интересней).